In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from torch.cuda.amp import GradScaler


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="/data/runs")
#!tensorboard --logdir=/data/runs #

In [3]:
GPU = torch.device("cuda:0")

In [4]:
SEED=1

In [5]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [6]:
torch.cuda.device_count()

1

In [7]:
START_EPOCH = 0

In [8]:
ARCH = 'resnet18'
EPOCHS = 2
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 100
TRAIN_BATCH=128
VAL_BATCH=128
WORKERS=4

In [9]:
TRAINDIR="/data/train2"
VALDIR="/data/val"

In [10]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [11]:
#torch.cuda.set_device('cpu')

In [12]:
cudnn.benchmark = True

In [13]:
scaler = GradScaler()

In [14]:
global_step_train = 0
global_step_val = 0

In [15]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step_train
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # compute gradient and do SGD step
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", losses.avg, global_step = global_step_train)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step_train)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step_train)
        global_step_train += 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [16]:
def validate(val_loader, model, criterion):
    global global_step_val
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            with torch.autocast(device_type='cuda', dtype=torch.float16):
                output = model(images)
                loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    writer.add_scalar("Loss/val", losses.avg, global_step = global_step_val)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step_val)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step_val)
    global_step_val += 1

    return top1.avg

In [17]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [18]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [19]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [20]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [21]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [22]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [23]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [24]:
#IMG_SIZE = 32 cinic size
IMG_SIZE = 224 # i think this is right for imagenet

In [25]:
# imagenet has 1000 classes
NUM_CLASSES = 1000

In [26]:
model = models.__dict__[ARCH]()

In [27]:
inf = model.fc.in_features
inf

512

In [28]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [29]:
model.cuda(GPU)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [30]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [31]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [32]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [33]:
transform_train = transforms.Compose([
    #transforms.RandomCrop(I),
    #transforms.Rescale(32),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
])

In [34]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [35]:
transform_val = transforms.Compose([
    #transforms.Rescale(32),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
])

In [36]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [37]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [38]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [39]:
best_acc1 = 0

In [40]:
for epoch in range(START_EPOCH, EPOCHS):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][    0/10010]	Time  3.043 ( 3.043)	Data  1.270 ( 1.270)	Loss 7.0402e+00 (7.0402e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Epoch: [0][  100/10010]	Time  0.077 ( 0.276)	Data  0.001 ( 0.182)	Loss 6.8359e+00 (7.0223e+00)	Acc@1   0.78 (  0.19)	Acc@5   2.34 (  0.89)
Epoch: [0][  200/10010]	Time  0.077 ( 0.271)	Data  0.001 ( 0.186)	Loss 6.8120e+00 (6.9221e+00)	Acc@1   0.00 (  0.30)	Acc@5   0.78 (  1.23)
Epoch: [0][  300/10010]	Time  0.077 ( 0.268)	Data  0.001 ( 0.186)	Loss 6.6579e+00 (6.8589e+00)	Acc@1   0.00 (  0.36)	Acc@5   1.56 (  1.45)
Epoch: [0][  400/10010]	Time  0.077 ( 0.265)	Data  0.001 ( 0.185)	Loss 6.6954e+00 (6.8076e+00)	Acc@1   0.00 (  0.41)	Acc@5   3.12 (  1.63)
Epoch: [0][  500/10010]	Time  0.077 ( 0.264)	Data  0.001 ( 0.184)	Loss 6.6104e+00 (6.7617e+00)	Acc@1   1.56 (  0.46)	Acc@5   5.47 (  1.90)
Epoch: [0][  600/10010]	Time  0.077 ( 0.263)	Data  0.001 ( 0.184)	Loss 6.4093e+00 (6.7164e+00)	Acc@1   0.78 (  0.51)	Acc@5   3.91 (  2.19)
Epoch: [0][  700/10010]	Tim